# Bedrock with LangChain - Explain/Interpret a code snippet or program 
> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

## Introduction

In this notebook we show you how to explain or interpret a given code snippet or program.

[LangChain](https://python.langchain.com/docs/get_started/introduction.html) is a framework for developing applications powered by language models. The key aspects of this framework allow us to augment the Large Language Models by chaining together various components to create advanced use cases.

In this notebook we will use the Bedrock API provided by LangChain. The prompt used in this example creates a custom LangChain prompt template for adding context to the code explain request. 

**Note:** *This notebook can be run within or outside of AWS environment.*

#### Context
In this notebook we will leverage the LangChain framework and explore Bedrock API with the help of `PromptTemplates`. `PrompTemplates` allow you to create generic shells which can be populated with information later and get model outputs based on different scenarios.

As part of this notebook we will explore the use of Amazon Bedrock integration within LangChain framework and how it could be used to generate or explain code with the help of `PromptTemplate`.

#### Pattern
We will simply provide the LangChain implementation of Amazon Bedrock API with an input consisting of a task, an instruction and an input for the model under the hood to generate an output without providing any additional example. The purpose here is to demonstrate how the powerful LLMs easily understand the task at hand and generate compelling outputs.

![](./images/code-interpret-langchain.png)

#### Use case
To demonstrate the code generation capability of models in Amazon Bedrock, let's take the use case of code explain.

#### Persona
You are Joe, a Java software developer, has been tasked to support a legacy C++ application for Vehicle Fleet Management. You need help to explain or interpret certain complex C++ code snippets as you are performing analyis to identify the business logic and potential problems with the code.

#### Implementation
To fulfill this use case, we will show you how you can Amazon Bedrock API with LangChain to explain C++ code snippets.


In [4]:
from IPython.core.display import HTML
from IPython.display import display_markdown, Markdown
import boto3

HTML("<script>Jupyter.notebook.kernel.restart()</script>")
boto3_bedrock = boto3.client('bedrock-runtime')

## Invoke the Bedrock LLM Model

We'll begin with creating an instance of Bedrock class from llms. This expects a `model_id` which is the ARN of the model available in Amazon Bedrock. 

Optionally you can pass on a previously created boto3 client as well as some `model_kwargs` which can hold parameters such as `temperature`, `topP`, `maxTokenCount` or `stopSequences` (more on parameters can be explored in Amazon Bedrock console).

Check [documentation](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids-arns.html) for Available text generation model Ids under Amazon Bedrock.

Note that different models support different `model_kwargs`.

In [5]:
from langchain_aws import ChatBedrockConverse


textgen_llm = ChatBedrockConverse(
    model_id="amazon.nova-lite-v1:0",
    client=boto3_bedrock,
    max_tokens=None,
    temperature=0.5
)

## Create a LangChain custom prompt template

By creating a template for the prompt we can pass it different input variables to it on every run. This is useful when you have to generate content with different input variables that you may be fetching from a database.

In [6]:
# Vehicle Fleet Management Code written in C++
sample_code = """
#include <iostream>
#include <string>
#include <vector>

class Vehicle {
protected:
    std::string registrationNumber;
    int milesTraveled;
    int lastMaintenanceMile;

public:
    Vehicle(std::string regNum) : registrationNumber(regNum), milesTraveled(0), lastMaintenanceMile(0) {}

    virtual void addMiles(int miles) {
        milesTraveled += miles;
    }

    virtual void performMaintenance() {
        lastMaintenanceMile = milesTraveled;
        std::cout << "Maintenance performed for vehicle: " << registrationNumber << std::endl;
    }

    virtual void checkMaintenanceDue() {
        if ((milesTraveled - lastMaintenanceMile) > 10000) {
            std::cout << "Vehicle: " << registrationNumber << " needs maintenance!" << std::endl;
        } else {
            std::cout << "No maintenance required for vehicle: " << registrationNumber << std::endl;
        }
    }

    virtual void displayDetails() = 0;

    ~Vehicle() {
        std::cout << "Destructor for Vehicle" << std::endl;
    }
};

class Truck : public Vehicle {
    int capacityInTons;

public:
    Truck(std::string regNum, int capacity) : Vehicle(regNum), capacityInTons(capacity) {}

    void displayDetails() override {
        std::cout << "Truck with Registration Number: " << registrationNumber << ", Capacity: " << capacityInTons << " tons." << std::endl;
    }
};

class Car : public Vehicle {
    std::string model;

public:
    Car(std::string regNum, std::string carModel) : Vehicle(regNum), model(carModel) {}

    void displayDetails() override {
        std::cout << "Car with Registration Number: " << registrationNumber << ", Model: " << model << "." << std::endl;
    }
};

int main() {
    std::vector<Vehicle*> fleet;

    fleet.push_back(new Truck("XYZ1234", 20));
    fleet.push_back(new Car("ABC9876", "Sedan"));

    for (auto vehicle : fleet) {
        vehicle->displayDetails();
        vehicle->addMiles(10500);
        vehicle->checkMaintenanceDue();
        vehicle->performMaintenance();
        vehicle->checkMaintenanceDue();
    }

    for (auto vehicle : fleet) {
        delete vehicle; 
    }

    return 0;
}
"""

In [7]:
from langchain.prompts import PromptTemplate

# Create a prompt template that has multiple input variables
multi_var_prompt = PromptTemplate(
    input_variables=["code", "programmingLanguage"], 
    template="""

Human: You will be acting as an expert software developer in {programmingLanguage}. 
You will explain the below code and highlight if there are any red flags or where best practices are not being followed.
<code>
{code}
</code>

Assistant:"""
)

# Pass in values to the input variables
prompt = multi_var_prompt.format(code=sample_code, programmingLanguage="C++")


### Explain C++ Code for Vehicle Fleet management using Amazon Bedrock and LangChain

In [8]:
response = textgen_llm.invoke(prompt)

code_explanation = response.content

display_markdown(Markdown(code_explanation))

This C++ code is a simple implementation of a vehicle fleet management system using polymorphism. Here's a detailed explanation and analysis of the code, highlighting any potential issues or areas where best practices are not being followed:

### Code Breakdown

1. **Includes and Namespace**:
   ```cpp
   #include <iostream>
   #include <string>
   #include <vector>
   ```
   These are standard C++ libraries for input/output, string manipulation, and dynamic arrays.

2. **Vehicle Class**:
   ```cpp
   class Vehicle {
   protected:
       std::string registrationNumber;
       int milesTraveled;
       int lastMaintenanceMile;
   ```
   This class serves as a base class for different types of vehicles. It has protected member variables for registration number, miles traveled, and the last maintenance mile.

   ```cpp
   public:
       Vehicle(std::string regNum) : registrationNumber(regNum), milesTraveled(0), lastMaintenanceMile(0) {}
   ```
   The constructor initializes the vehicle wi

## Summary

To conclude we learnt that invoking the LLM without any context might not yield the desired results. By adding context and further using the prompt template to constrain the output from the LLM we are able to successfully get our desired output